In [48]:
import pandas as pd
from tqdm import tqdm

In [49]:
MIMICtable = pd.read_csv("/data/localhost/taufiq/export-dir/MIMICtable-1hourly.csv")
MIMICtable['bloc'] = MIMICtable.bloc.astype(int)
MIMICtable['icustay_id'] = MIMICtable.icustay_id.astype(int)

In [50]:
MIMICtable.head()

,bloc,icustay_id,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_1hourly,output_total,output_1hourly,cumulated_balance
0,1,3,7.245400e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.663900,599.999991,0.0,0.0,6267.0,0.0,9490.0,4705.0,-3223.0
1,2,3,7.245403e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.672340,339.999995,0.0,0.0,6277.0,10.0,14095.0,4605.0,-7818.0
2,3,3,7.245407e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.649789,339.999995,0.0,0.0,6297.0,20.0,18540.0,4445.0,-12243.0
3,4,3,7.245410e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.634454,339.999995,0.0,0.0,6317.0,20.0,22845.0,4305.0,-16528.0
4,5,3,7.245414e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.645714,497.499993,0.0,0.0,6337.0,20.0,27050.0,4205.0,-20713.0


In [84]:
icustayids = set(MIMICtable['icustay_id'])

In [52]:
len(icustayids)

17161

In [53]:
MIMICtable_filtered = MIMICtable[MIMICtable['bloc']<12]

In [54]:
len(MIMICtable_filtered)

131590

In [55]:
len(MIMICtable)

610551

In [88]:
processed_data = pd.DataFrame()
filtered_icustayids = []
for icustay_id in tqdm(icustayids):
    data = MIMICtable_filtered[MIMICtable_filtered['icustay_id'] == icustay_id]
    if len(data) > 0 and set(data['bloc']) == set(range(1,len(data)+1)):
        filtered_icustayids.append(icustay_id)
        rows = pd.DataFrame([[-1]*len(data.columns)]*(11-len(data)), columns=data.columns)
        data = data.append(rows, ignore_index=True)
        data['icustay_id'] = icustay_id
        processed_data = processed_data.append(data, ignore_index=True)

100%|██████████| 17161/17161 [01:49<00:00, 156.80it/s]


In [90]:
processed_data.head()

,bloc,icustay_id,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_1hourly,output_total,output_1hourly,cumulated_balance
0,1,65537,7.139723e+09,0.0,22799.40478,4.0,1.0,0.0,0.0,0.0,...,0.0,0.687500,363.888889,0.18,0.18,4050.487733,29.994183,7235.0,3605.0,-3184.512267
1,2,65537,7.139727e+09,0.0,22799.40478,4.0,1.0,0.0,0.0,0.0,...,0.0,0.602564,430.555556,0.00,0.00,4565.895400,515.407667,10760.0,3525.0,-6194.104600
2,3,65537,7.139731e+09,0.0,22799.40478,4.0,1.0,0.0,0.0,0.0,...,0.0,0.751456,402.222222,0.00,0.00,5113.772100,547.876700,14235.0,3475.0,-9121.227900
3,4,65537,7.139734e+09,0.0,22799.40478,4.0,1.0,0.0,0.0,0.0,...,0.0,0.804762,275.000000,0.00,0.00,7210.916100,2097.144000,17630.0,3395.0,-10419.083900
4,5,65537,7.139738e+09,0.0,22799.40478,4.0,1.0,0.0,0.0,0.0,...,0.0,0.804245,283.000000,0.00,0.00,7730.749683,519.833583,20785.0,3155.0,-13054.250317


In [91]:
processed_data.to_csv("/data/localhost/taufiq/export-dir/MIMIC-1hourly-processed.csv", index=False)